In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
!pip install mediapipe

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False  
    results = model.process(image)  
    image.flags.writeable = True  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  
    return image, results

def draw_landmarks(image, results):
    # pose landmarks
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    # left hand landmarks
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    # right hand landmarks
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


In [ ]:
def draw_styled_landmarks(image, results):'
    # pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh, rh])


In [ ]:

!pip install opencv-python


import cv2
import os


VIDEO_FOLDER_PATH = '/content/drive/MyDrive/train' 


FRAME_OUTPUT_PATH = '/content/path_to_save_extracted_frames'  


os.makedirs(FRAME_OUTPUT_PATH, exist_ok=True)


for video_file in os.listdir(VIDEO_FOLDER_PATH):
    video_path = os.path.join(VIDEO_FOLDER_PATH, video_file)

    if video_file.endswith(('.mp4', '.avi', '.mov', '.mkv')):

       
        video_name = os.path.splitext(video_file)[0]  
        video_output_dir = os.path.join(FRAME_OUTPUT_PATH, video_name)
        os.makedirs(video_output_dir, exist_ok=True)

        cap = cv2.VideoCapture(video_path)

       
        if not cap.isOpened():
            print(f"Error: Could not open video {video_file}.")
            continue

        
        frame_idx = 0

        while cap.isOpened():
            ret, frame = cap.read()  

            if not ret:
                break  

            
            frame_filename = f"{video_name}_frame{frame_idx:04d}.jpg"  
            frame_output_path = os.path.join(video_output_dir, frame_filename)

            
            cv2.imwrite(frame_output_path, frame)

            frame_idx += 1  

       
        cap.release()

print(f"Frame extraction complete. All frames saved to {FRAME_OUTPUT_PATH}.")

Frame extraction complete. All frames saved to /content/path_to_save_extracted_frames.


In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
from tensorflow.keras.utils import to_categorical


DATA_PATH = '/content/path_to_save_extracted_frames'
actions = np.array(["absent", "baby", "bacteria", "cabbage", "call", "dark", "face", "fall"]) 

sequence_length = 30
label_map = {label: num for num, label in enumerate(actions)}


mp_holistic = mp.solutions.holistic


sequences, labels = [], []


def adjust_brightness(image, brightness=30):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = np.clip(v.astype(np.int32) + brightness, 0, 255).astype(np.uint8)
    final_hsv = cv2.merge((h, s, v))
    return cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

def augment_image(image):
    aug_images = []

   
    aug_images.append(image)

   
    aug_images.append(cv2.flip(image, 1))

    
    aug_images.append(adjust_brightness(image, 30))
    aug_images.append(adjust_brightness(image, -30))

    
    h, w = image.shape[:2]
    center = (w // 2, h // 2)

    
    M = cv2.getRotationMatrix2D(center, 10, 1.0)
    aug_images.append(cv2.warpAffine(image, M, (w, h)))

    M = cv2.getRotationMatrix2D(center, -10, 1.0)
    aug_images.append(cv2.warpAffine(image, M, (w, h)))

    return aug_images

def process_frames_with_augmentation(image, holistic):
   
    keypoints = []

    augmented_images = augment_image(image)

    for aug_image in augmented_images:
      
        image, results = mediapipe_detection(aug_image, holistic)
        keypoints.append(extract_keypoints(results))

    return keypoints

for action in actions:
    action_dir = os.path.join(DATA_PATH, action)
    available_frames = sorted([f for f in os.listdir(action_dir) if f.endswith('.jpg')])  

    
    augmented_frame_count = len(available_frames) * 6  


    no_sequences = augmented_frame_count // sequence_length  

    print(f"Processing action: {action}, Available frames: {augmented_frame_count}, Sequences to extract: {no_sequences}")

    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):

            frame_filename = available_frames[sequence * sequence_length // 6 + frame_num // 6] 
            frame_path = os.path.join(action_dir, frame_filename)

            if os.path.isfile(frame_path):
                image = cv2.imread(frame_path)

                with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
                    keypoints = process_frames_with_augmentation(image, holistic)
                    window.extend(keypoints)

        if len(window) == sequence_length * 6:  
            sequences.append(window)
            labels.append(label_map[action])
X = np.array(sequences)
y = to_categorical(labels).astype(int)

print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")


Processing action: absent, Available frames: 372, Sequences to extract: 12


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing action: baby, Available frames: 312, Sequences to extract: 10
Processing action: bacteria, Available frames: 474, Sequences to extract: 15
Processing action: cabbage, Available frames: 372, Sequences to extract: 12
Processing action: call, Available frames: 282, Sequences to extract: 9
Processing action: dark, Available frames: 402, Sequences to extract: 13
Processing action: face, Available frames: 276, Sequences to extract: 9
Processing action: fall, Available frames: 282, Sequences to extract: 9
Data shape: (89, 180, 258)
Labels shape: (89, 8)


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.array(sequences)
y = to_categorical(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")




X_train shape: (66, 180, 258)
y_train shape: (66, 8)
X_test shape: (23, 180, 258)
y_test shape: (23, 8)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam


model = Sequential()

model.add(LSTM(256, return_sequences=True, input_shape=(180, 258),
               kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005)))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.0005), recurrent_regularizer=l2(0.0005)))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.0005)))
num_classes = 8
model.add(Dense(num_classes, activation='softmax'))
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                       │ (None, 180, 256)            │         527,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 180, 256)            │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (None, 180, 128)            │         197,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 180, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_12               │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 780,360 (2.98 MB)

 Trainable params: 779,464 (2.97 MB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np

# Compute class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

# Define optimizer with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), class_weight=class_weights, callbacks=[early_stopping])


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 944ms/step - categorical_accuracy: 0.1213 - loss: 2.7368 - val_categorical_accuracy: 0.1304 - val_loss: 2.5718
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - categorical_accuracy: 0.2322 - loss: 2.5145 - val_categorical_accuracy: 0.1304 - val_loss: 2.5715
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 778ms/step - categorical_accuracy: 0.1992 - loss: 2.4770 - val_categorical_accuracy: 0.1304 - val_loss: 2.5714
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 854ms/step - categorical_accuracy: 0.2965 - loss: 2.2748 - val_categorical_accuracy: 0.1304 - val_loss: 2.5700
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 977ms/step - categorical_accuracy: 0.2430 - loss: 2.3990 - val_categorical_accuracy: 0.1304 - val_loss: 2.5691
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 760ms/step - categorical_accuracy: 0.2274 - loss: 2.4437 - val_categorical_accuracy: 0.1304 - val_loss: 2.5672
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 800ms/step - categorical_accuracy: 0.2713 - loss: 2.3475

In [ ]:
model.save('model.h5')